In [5]:
import sys
sys.path.append("/mnt/shared/Huggingface/sharedcode/Stegastamp_CR")
sys.path.append("/data/shared/Huggingface/sharedcode/Stegastamp_CR")
from utils.helpers import *
from dataset import CustomImageFolder
from torchvision import transforms
from functools import partial
import torch
from functions import *
from clustering_function import text_low,text_mid,text_high
from torchvision.datasets import ImageFolder
from clustering_function import cluster
from sklearn.manifold import TSNE
import csv


mi = "" #mi
# forge setting 
text_num = 4          # 1 4
random = False
transform_pipe = [
    transforms.Resize((128,128)),
    transforms.ToTensor(),
]
normalize_ = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
un_normalize_ = transforms.Normalize([-1,-1,-1],[2,2,2])

set_seeds(2024)
device = torch.device("cuda:0")

for model_choice in ["stega"]:
    for data_choice in ["CelebA","COCO"]:
        data_dir = {"COCO":"/data/shared/coco2017/test2017","CelebA":"/data/shared/deepfake/CelebA-HQ/val"}
        ckp_dir = f"/data/shared/Huggingface/sharedcode/Stegastamp_Train/{mi}weights/{data_choice}/{model_choice}"

        if model_choice == 'hidden':
            transform_pipe.append(transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]))
        transform = transforms.Compose(transform_pipe)
        if data_choice == "CelebA":
            ds = ImageFolder(data_dir[data_choice],transform=transform)
        else:
            ds = CustomImageFolder(data_dir[data_choice],transform,num=1000)
        message_len = 100 if model_choice == "hidden" else 100

        # functions
        get_wmimages = partial(get_wmimages,ds=ds,device=device,model_choice=model_choice)
        get_images = partial(get_images,ds=ds,device=device)
        extract_message = partial(extract_message,device=device,model_choice=model_choice)

        # tsne forge setting
        tolerant = cal_tolerant(message_len)
        batch_size = 16
        alpha = 1
        if text_num == 1:
            attack_num = 1       # 1 30
        else:
            attack_num = 30
        sample_num = 5
        use_vae = False

        ckp_list = ["emperical"]
        # ckp_list = sorted(os.listdir(ckp_dir))
        
        if f"{mi}OO" in ckp_list:
            ckp_list.remove(f"{mi}OO")
            ckp_list.insert(0,f"{mi}OO")

        
        for gpname,text_strings in zip(["low","mid","high"],[text_low,text_mid,text_high]):
            csv_file = f"../forged_results/text_distance/{data_choice}_{model_choice}_{gpname}_{text_num}_{mi}_alpha{alpha}.csv"
            text_list = [generate_message(message_len,t,1) for t in text_strings]
            for op in ckp_list:
                if op == "OO":
                    ckp_path = os.path.join(ckp_dir,op,'epoch_499_state100.pth')
                elif op == "emperical":
                    ckp_path = os.path.join(ckp_dir,op,'epoch_499_state.pth')
                else:
                    ckp_path = os.path.join(ckp_dir,op,'epoch_99_state.pth')
                encoder,decoder = load_weights(ckp_path,model_choice,message_len)
                encoder.eval(),decoder.eval()
                # get a watermarked image as target 
                target_image,target_residual = get_wmimages(text=text_list[-1],image_i=[666],encoder=encoder)
                # watermarked images by different text
                bit_acc_list = []
                acc_list = []
                for j in range(0,text_num*sample_num,text_num):
                    wm_images,residual_predictions = zip(*[get_wmimages(text=t,image_i=range((i+j)*attack_num,(i+j+1)*attack_num),encoder=encoder) for i,t in enumerate(text_list)])
                    if use_vae:
                        residual_predictions = [get_residual_prediction(wm_images_i,batch_size,device,method="diffusion") for wm_images_i in wm_images]
                    wm_images = list(wm_images)
                    residual_predictions = list(residual_predictions)
                    residual_predictions.append(target_residual)
                    # TSNE
                    if text_num != 1:
                        data = torch.cat(residual_predictions,dim=0).detach().cpu()
                        flattened_data = data.view((text_num)*attack_num+1,-1).numpy()
                        tsne = TSNE(n_components=2, random_state=10000)
                        reduced_data = tsne.fit_transform(flattened_data)

                        # cluster
                        cluster_labels,cluster_centers = cluster(reduced_data,text_num,"km")
                        # print(cluster_labels.shape,cluster_centers.shape)
                        target_label = cluster_labels[-1]
                        attack_samples = data[cluster_labels == target_label]

                        # rescale
                        attack_samples = un_normalize_(attack_samples)  # 0-1
                        attack_sample = attack_samples.mean(axis=0)
                        attack_sample = normalize_(attack_sample) # -1,1
                        attack_pattern = attack_sample.unsqueeze(0).to(device)
                    
                    # print(attack_samples.shape)
                    else:
                        attack_sample = residual_predictions[-1]
                        attack_pattern = attack_sample.to(device)

                    # forge
                    original_images = get_images(image_i=range(900,1000)).to(device)
                    # original_images = to0255(original_images)
                    # attack_pattern = to0255(attack_pattern)

                    forged_image = (original_images + alpha * attack_pattern)
                    # print(f"attack_pattern:{attack_pattern}")
                    # print(f"original_images:{original_images}")
                    forged_image = torch.clamp(forged_image, min=-1, max=1)

                    forged_prediction,_ = extract_message(image_tensor=forged_image,decoder=decoder,model_choice=model_choice)
                    gt_text = text_list[-1].repeat(forged_prediction.size(0),1).to(device)
                    difference = (forged_prediction != gt_text).float()
                    correct = (difference.sum(dim=1)<=tolerant).float()
                    acc = correct.mean(dim=0).item()
                    bitwise_accuracy = (1.0 - difference.mean(dim=1))
                    bitwise_accuracy = torch.mean(bitwise_accuracy)
                    bit_acc_list.append(bitwise_accuracy.item())
                    acc_list.append(acc)
                u_bit_acc,std_bit_acc = calculate_mean_and_variance(bit_acc_list)
                u_acc,std_acc = calculate_mean_and_variance(acc_list)
                print(f"model:{op}-forged_bit_acc:{u_bit_acc,std_bit_acc} acc:{u_acc,std_acc}")
                csv_dict = {"model":op,"u_bit_acc":u_bit_acc,"std_bit_acc":std_bit_acc,"u_acc":u_acc,"std_acc":std_acc}
                with open(csv_file, mode='a', newline='') as file:
                    fieldnames = csv_dict.keys()
                    writer = csv.DictWriter(file, fieldnames=fieldnames)
                    if file.tell() == 0:
                        writer.writeheader()
                    writer.writerow(csv_dict)



dd
uncompatible keys: <All keys matched successfully>
model:emperical-forged_bit_acc:(1.0, 0.0) acc:(1.0, 0.0)
dd
uncompatible keys: <All keys matched successfully>
model:emperical-forged_bit_acc:(1.0, 0.0) acc:(1.0, 0.0)
dd
uncompatible keys: <All keys matched successfully>
model:emperical-forged_bit_acc:(1.0, 0.0) acc:(1.0, 0.0)
dd
uncompatible keys: <All keys matched successfully>
model:emperical-forged_bit_acc:(0.9238199949264526, 0.00018011396433706748) acc:(1.0, 0.0)
dd
uncompatible keys: <All keys matched successfully>
model:emperical-forged_bit_acc:(0.9902400255203248, 5.503843033238809e-07) acc:(1.0, 0.0)
dd
uncompatible keys: <All keys matched successfully>
model:emperical-forged_bit_acc:(0.9897799730300904, 1.8962239579423112e-07) acc:(1.0, 0.0)
